In [19]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import numpy as np

In [20]:
# Загрузка предварительно обученной модели для детекции лиц
face_cascade = cv2.CascadeClassifier('path/to/haarcascade_frontalface_default.xml')

In [21]:
# Загрузка предварительно обученной модели для классификации жестов рук (например, ResNet)
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 10)

In [1]:
# Загрузка обученных весов модели для классификации жестов рук
model.load_state_dict(torch.load('path/to/model_weights.pth'))


In [2]:
# Загрузка датасета для меток жестов рук
with open('path/to/leapgestrecog/leapGestRecog/leapGestRecogMapping.txt') as f:
    gestures = f.readlines()
gestures = [gesture.strip() for gesture in gestures]



In [7]:
# Установка устройства для вычислений
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# Предобработка изображения
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
# Захват видеопотока с камеры
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

In [3]:
# Обнаружение лица на кадре
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

In [4]:
  # Обработка каждого обнаруженного лица
for (x, y, w, h) in faces:
    face = frame[y:y+h, x:x+w]
    image_tensor = transform(face).unsqueeze(0).to(device)

In [5]:
# Классификация жестов рук на изображении
with torch.no_grad():
    outputs = model(image_tensor)
    probabilities = torch.softmax(outputs, dim=1)[0]
    predicted_class = torch.argmax(probabilities).item()

In [6]:
  # Вывод результата классификации на кадр
gesture_label = gestures[predicted_class]
label = f"Gesture: {gesture_label}"
cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

In [7]:
  # Отображение кадра

while(True):
    ret, frame = cap.read()
    cv2.imshow('Gesture Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [18]:
# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()